In [1]:
from nlptoolkits.FirmLevelRiskKits import DictionaryT, PreprocessT
from nlptoolkits.SmallKits import IOHandlerT

We make the basic classes for further use

In [2]:
internation_tax_dictionary = IOHandlerT.file_to_list(
    './input_data/annotatedsent_Principles_of_International_Taxation.txt',
    charset_error_encoding='utf-8'
)[:1000]

tweets_texture = IOHandlerT.file_to_list(
    './input_data/tweets_origin.txt',
    charset_error_encoding='utf-8'
)[:1000]

# make a class to clean the data(dict)
corenlp_preprocess_cls = PreprocessT.NgramDataPreprocessor()

# make a class to build the dictionary
corenlp_ngram_dict_cls = DictionaryT.NgramDictionaryBuilder()

In [3]:
internation_tax_dictionary[:3]

In [4]:
tweets_texture[:3]

Part to make Ngram dictionary

In [5]:
international_tax_tfidf = corenlp_ngram_dict_cls.n_gramizer_dictionary_builder(internation_tax_dictionary, scorer = 'tfidf', 
                                                                               remove_ngram_postags_combinations=None
                                                                               )

In [6]:
international_tax_tfidf

You can remove words by remove phrases which are in another dictionary, but we do not do here now.

We now clean the tweets data quickly by using the nltk quick cleaner

In [16]:
PreprocessT.naive_nltk_annotator(
    tweets_texture
)[0]

In [17]:
corenlp_preprocess_cls.clean_annotated_texture_list(PreprocessT.naive_nltk_annotator(
    tweets_texture
), processes=5)[0]